# Deliverable 1

In [2]:
# Import Dependencies
import pandas as pd
import requests
import random
import numpy as np

# Import API keys
from config import weather_api_key

In [3]:
# Generate 2000 random latitudes and longitudes
lats = np.random.uniform(-90.000, 90.000, size = 2000)
lngs = np.random.uniform(-180.000, 180.000, size = 2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list (unzip)
coordinates = list(lat_lngs)

In [5]:
# Display the 2000 coordinates
coordinates

[(-22.405627478073484, -83.62773488215487),
 (73.69875966486347, -47.80760898236346),
 (-29.11590374371977, -70.74100405091289),
 (64.96170450809282, -70.74924010892177),
 (-4.9908065637393975, 81.89766618233472),
 (2.2426918206036675, -109.72667426868894),
 (66.09738116797729, 70.11899916541202),
 (72.43917150225238, 57.714595260990905),
 (22.266019400125188, -4.1441344763908035),
 (-66.8204362630547, -9.999081104246443),
 (44.140737893838974, 93.171168586774),
 (23.07123858454588, 177.02100641778833),
 (-64.97307298688867, 19.645578547318422),
 (-18.147428528292608, 79.64065376621289),
 (-14.432795450493217, -118.47906631875645),
 (5.571005435112141, -21.713512240776396),
 (15.94745716889426, -134.0703116486655),
 (82.74002274341149, -6.478133845362379),
 (52.25959779633095, -64.10612655182521),
 (-55.75515881817646, 171.16878303244476),
 (-80.07798045802653, -1.6907087224712711),
 (85.74749115135492, 126.12610780753943),
 (78.21368537732249, -3.095514303507514),
 (67.94293728835092,

In [6]:
# Get the nearest City using the Citipy module
#import citipy
from citipy import citipy

# Create a list for the cities
cities = []

# identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the number of cities.
len(cities)

741

In [7]:
# Perform API call to OpenWeatherMap to get:latitiude and longitude, max temp, 
# percent humidity, percent cloudiness, wind speed, and weather description

# import the API key
from config import weather_api_key

# Define Base URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

In [8]:
# Test CITY API call
city_url = url + "&q=" + "London"
test_weather = requests.get(city_url).json()


In [9]:
test_weather

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 48.78,
  'feels_like': 44.04,
  'temp_min': 46.85,
  'temp_max': 50.27,
  'pressure': 1010,
  'humidity': 82},
 'visibility': 10000,
 'wind': {'speed': 11.5, 'deg': 220},
 'clouds': {'all': 75},
 'dt': 1643942513,
 'sys': {'type': 2,
  'id': 2019646,
  'country': 'GB',
  'sunrise': 1643960063,
  'sunset': 1643993664},
 'timezone': 0,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [10]:
test_weather["weather"][0]["description"]

'broken clouds'

In [11]:
test_weather["sys"]["country"]

'GB'

In [12]:
# Import the time library and the datetime modeul from the datetime library
import time
from datetime import datetime

In [13]:
# Create list to hold the weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval      ")
print("------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >=50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities

    try:
        #Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #Append the city information into city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_description,
                          "Date": city_date})
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete.
print("--------------------------")
print("Data Retieval Complete     ")
print("--------------------------")

Beginning Data Retrieval      
------------------------------
Processing Record 1 of Set 1 | marcona
City not found. Skipping...
Processing Record 2 of Set 1 | ilulissat
Processing Record 3 of Set 1 | vallenar
Processing Record 4 of Set 1 | iqaluit
Processing Record 5 of Set 1 | hithadhoo
Processing Record 6 of Set 1 | ixtapa
Processing Record 7 of Set 1 | yar-sale
Processing Record 8 of Set 1 | amderma
City not found. Skipping...
Processing Record 9 of Set 1 | taoudenni
Processing Record 10 of Set 1 | cape town
Processing Record 11 of Set 1 | hami
Processing Record 12 of Set 1 | butaritari
Processing Record 13 of Set 1 | bredasdorp
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | bubaque
Processing Record 16 of Set 1 | hilo
Processing Record 17 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 18 of Set 1 | havre-saint-pierre
Processing Record 19 of Set 1 | kaitangata
Processing Record 20 of Set 1 | hermanus
Processing Record 21 of Set 1 

Processing Record 37 of Set 4 | nuoro
Processing Record 38 of Set 4 | san andres
Processing Record 39 of Set 4 | sola
Processing Record 40 of Set 4 | longyearbyen
Processing Record 41 of Set 4 | maralal
Processing Record 42 of Set 4 | mahebourg
Processing Record 43 of Set 4 | altay
Processing Record 44 of Set 4 | cherskiy
Processing Record 45 of Set 4 | mecca
Processing Record 46 of Set 4 | leningradskiy
Processing Record 47 of Set 4 | vaitupu
City not found. Skipping...
Processing Record 48 of Set 4 | ostrovnoy
Processing Record 49 of Set 4 | scarborough
Processing Record 50 of Set 4 | rocha
Processing Record 1 of Set 5 | devils lake
Processing Record 2 of Set 5 | meulaboh
Processing Record 3 of Set 5 | tirano
Processing Record 4 of Set 5 | cockburn town
Processing Record 5 of Set 5 | nabire
Processing Record 6 of Set 5 | zemio
Processing Record 7 of Set 5 | salalah
Processing Record 8 of Set 5 | lavrentiya
Processing Record 9 of Set 5 | faanui
Processing Record 10 of Set 5 | huntervi

Processing Record 31 of Set 8 | monrovia
Processing Record 32 of Set 8 | toliary
City not found. Skipping...
Processing Record 33 of Set 8 | kosh-agach
Processing Record 34 of Set 8 | santiago
Processing Record 35 of Set 8 | melton
Processing Record 36 of Set 8 | gubkinskiy
Processing Record 37 of Set 8 | beberibe
Processing Record 38 of Set 8 | okha
Processing Record 39 of Set 8 | tazovskiy
Processing Record 40 of Set 8 | aldan
Processing Record 41 of Set 8 | russell
Processing Record 42 of Set 8 | borogontsy
Processing Record 43 of Set 8 | luanda
Processing Record 44 of Set 8 | palmer
Processing Record 45 of Set 8 | port-gentil
Processing Record 46 of Set 8 | portland
Processing Record 47 of Set 8 | qasigiannguit
Processing Record 48 of Set 8 | belle fourche
Processing Record 49 of Set 8 | hervey bay
Processing Record 50 of Set 8 | chapais
Processing Record 1 of Set 9 | sangar
Processing Record 2 of Set 9 | kizema
Processing Record 3 of Set 9 | carballo
Processing Record 4 of Set 9 |

Processing Record 18 of Set 12 | lady frere
Processing Record 19 of Set 12 | kamen-rybolov
Processing Record 20 of Set 12 | jacareacanga
Processing Record 21 of Set 12 | ngukurr
City not found. Skipping...
Processing Record 22 of Set 12 | mahadday weyne
City not found. Skipping...
Processing Record 23 of Set 12 | nizhniy odes
Processing Record 24 of Set 12 | butterworth
Processing Record 25 of Set 12 | coracao de jesus
Processing Record 26 of Set 12 | dvinskoy
Processing Record 27 of Set 12 | inuvik
Processing Record 28 of Set 12 | wum
Processing Record 29 of Set 12 | yelabuga
Processing Record 30 of Set 12 | banjar
Processing Record 31 of Set 12 | camacha
Processing Record 32 of Set 12 | barabai
Processing Record 33 of Set 12 | slave lake
Processing Record 34 of Set 12 | araouane
Processing Record 35 of Set 12 | aguimes
Processing Record 36 of Set 12 | xuanhua
Processing Record 37 of Set 12 | ambodifototra
City not found. Skipping...
Processing Record 38 of Set 12 | soyo
Processing Re

In [14]:
#  Convert the array of dictionaires to a Pandas DataFrame
city_data_df= pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,Ilulissat,GL,69.2167,-51.1000,19.42,58,100,16.11,overcast clouds,2022-02-04 02:49:49
1,Vallenar,CL,-28.5708,-70.7581,65.66,62,0,2.57,clear sky,2022-02-04 02:49:49
2,Iqaluit,CA,63.7506,-68.5145,-9.67,64,75,4.61,light snow,2022-02-04 02:49:00
3,Hithadhoo,MV,-0.6000,73.0833,81.64,74,100,10.09,overcast clouds,2022-02-04 02:46:14
4,Ixtapa,MX,20.7000,-105.2000,65.07,81,73,2.19,broken clouds,2022-02-04 02:49:50
5,Yar-Sale,RU,66.8333,70.8333,-2.74,98,99,5.08,overcast clouds,2022-02-04 02:49:50
6,Taoudenni,ML,22.6783,-3.9836,61.50,20,36,10.07,scattered clouds,2022-02-04 02:49:50
7,Cape Town,ZA,-33.9258,18.4232,62.28,73,0,5.01,clear sky,2022-02-04 02:47:29
8,Hami,CN,42.8000,93.4500,24.12,33,14,1.77,few clouds,2022-02-04 02:49:51
9,Butaritari,KI,3.0707,172.7902,81.90,76,17,14.52,light rain,2022-02-04 02:48:42


In [15]:
# Create the output file (CSV)
#output_data_file = "Weather_Database/WeatherPy_Database.csv"
output_data_file = "WeatherPy_Database.csv"
# Export the City_data into a CSV
city_data_df.to_csv(output_data_file, index_label = "City_ID")